# 00 - SemanticKITTI 前処理ノートブック

このノートブックでは、SemanticKITTI データセットをダウンロードし、PointNet 用に前処理（CSV 変換）を行います。

## SemanticKITTI のダウンロード

SemanticKITTI は下記より入手可能です：[http://www.semantic-kitti.org/dataset.html](http://www.semantic-kitti.org/dataset.html)

以下は `wget` を使って sequence 00 のみをダウンロードする例です。※データサイズが大きいため実行環境の容量に注意してください。

In [ ]:
!mkdir -p semantic_kitti
!cd semantic_kitti && \
        wget -N http://semantic-kitti.org/dataset/sequences.zip && \
        unzip -n sequences.zip && \
        rm sequences.zip

In [ ]:
!ls semantic_kitti/sequences/00/velodyne | head

## 必要なライブラリのインポート

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

## ファイル読み込みと前処理（.bin → CSV）

In [ ]:
root_dir = "semantic_kitti/sequences/00"
velodyne_dir = os.path.join(root_dir, "velodyne")
label_dir = os.path.join(root_dir, "labels")
out_dir = "data/preprocessed"
os.makedirs(out_dir, exist_ok=True)

bin_files = sorted([f for f in os.listdir(velodyne_dir) if f.endswith(".bin")])
print(f"{len(bin_files)} .bin files found.")

In [ ]:
for fname in tqdm(bin_files[:50]):  # デモ用に先頭50ファイルのみ
    bin_path = os.path.join(velodyne_dir, fname)
    label_path = os.path.join(label_dir, fname.replace(".bin", ".label"))

    pc = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)[:, :3]
    labels = np.fromfile(label_path, dtype=np.uint32) & 0xFFFF

    df = pd.DataFrame(pc, columns=['x', 'y', 'z'])
    df['label'] = labels
    df.to_csv(os.path.join(out_dir, fname.replace(".bin", ".csv")), index=False)

In [ ]:
print(f"変換後のCSVファイル数: {len(os.listdir(out_dir))}")